In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import pandas as pd
import pylhe
from tqdm import tqdm
import itertools

In [2]:
def f(deltaeta, deltaphi):
    return np.sqrt(deltaeta**2 + deltaphi**2)
def function(f_Att, cs):
    # Define Tree
    tree_electron_Att = f_Att['Delphes;1/Electron']
    tree_muon_Att = f_Att['Delphes;1/Muon']
    tree_jet_Att = f_Att['Delphes;1/Jet']
    tree_Emiss_Att = f_Att['Delphes;1/MissingET']
    
    # Define Branches
    Att_electron = tree_electron_Att.arrays(['Electron.Charge',
                                            'Electron.PT',
                                            'Electron.Eta',
                                            'Electron.Phi'], library='ak')
    Att_muon = tree_muon_Att.arrays(['Muon.Charge',
                                    'Muon.PT',
                                    'Muon.Eta',
                                    'Muon.Phi'], library='ak')
    Att_jet = tree_jet_Att.arrays(['Jet.Mass',
                                   'Jet.BTag',
                                   'Jet.PT',
                                   'Jet.Eta', 
                                   'Jet.Phi'], library='ak')
    Att_Emiss = tree_Emiss_Att.arrays(['MissingET.MET',
                                       'MissingET.Phi'], library='ak')
    
    # Define Variables
    EventSize = len(Att_jet['Jet.Mass'])
    jet_sort_pt = ak.sort(Att_jet['Jet.PT'], axis=-1, ascending=False)
    jet_num = ak.num(Att_jet['Jet.PT'], axis=-1)
    lepton_sort_pt = ak.sort(ak.concatenate([Att_electron['Electron.PT'], Att_muon['Muon.PT']], axis=-1), axis=-1, ascending=False)
    lepton_charge = ak.Array.tolist(ak.concatenate([Att_electron['Electron.Charge'], Att_muon['Muon.Charge']], axis=-1))
    lepton_eta = ak.concatenate([Att_electron['Electron.Eta'], Att_muon['Muon.Eta']], axis=-1)
    lepton_phi = ak.concatenate([Att_electron['Electron.Phi'], Att_muon['Muon.Phi']], axis=-1)
    bjet_pt = Att_jet['Jet.PT']*Att_jet['Jet.BTag']
    bjet_eta = Att_jet['Jet.Eta']*Att_jet['Jet.BTag']
    bjet_phi = Att_jet['Jet.Phi']*Att_jet['Jet.BTag']
    three_lepton = ak.where(ak.num(lepton_sort_pt)>=3)
    
    # At least three leptons and three b-jets
    event_signal = []
    for i in tqdm(three_lepton[0]): #Awkward Array has a length of 1
        b_pt = bjet_pt[i]
        num_bjet = len(b_pt[ak.where(b_pt)])
        if num_bjet >= 3:
            event_signal.append(i)
        
    # Transverse Momentum of leading and subleading lepton & Eta for Leptons
    event_lep = []
    for i in tqdm(event_signal):
        lep_eta = lepton_eta[i]
        lep_where = ak.where(np.abs(lep_eta)<2.5, 0, lep_eta)
        if lepton_sort_pt[i][2] > 25 and ak.count_nonzero(lep_where) == 0:
            event_lep.append(i)
        else:
            continue
    
    # Transverse Momentum of leading and subleading b-jet & Eta for b-jets
    event_jet = []
    for i in tqdm(event_lep):
        b_pt, b_eta  = bjet_pt[i], bjet_eta[i]
        b_pt = b_pt[ak.where(b_pt)]
        b_pt = ak.sort(b_pt, ascending=False)
        bjet_where = ak.where(np.abs(b_eta)<2.5, 0, b_eta)
        if b_pt[2] > 20 and ak.count_nonzero(bjet_where) == 0:
            event_jet.append(i)
        else:
            continue
    
    # Emiss
    event_emiss = []
    for i in tqdm(event_jet):
        if Att_Emiss['MissingET.MET'][i] > 30:
            event_emiss.append(i)
        else:
            continue
 
    # delta R between b-jets and leptons, leptons and leptons, b-jets and b-jets
    event_deltaR_bl = []
    for i in tqdm(event_emiss):
        lep_eta, lep_phi, b_eta, b_phi = lepton_eta[i], lepton_phi[i], bjet_eta[i], bjet_phi[i]
        b_eta, b_phi = b_eta[ak.where(b_eta)], b_phi[ak.where(b_phi)]

        deltaeta_bl = np.array([i-j for i in b_eta for j in lep_eta])
        deltaphi_bl = np.abs([i-j for i in b_phi for j in lep_phi])
        deltaphi_bl = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_bl])
    
        if min(f(deltaeta_bl, deltaphi_bl)) > 0.4:
            event_deltaR_bl.append(i)
    
    event_deltaR_bb = []
    for i in tqdm(event_deltaR_bl):
        b_eta, b_phi = bjet_eta[i], bjet_phi[i]
        b_eta, b_phi = b_eta[ak.where(b_eta)], b_phi[ak.where(b_phi)]
        
        m = b_eta - b_eta[:, np.newaxis]
        deltaeta_bb = m[np.triu_indices(len(m[0]), k = 1)]
        m = b_phi - b_phi[:, np.newaxis]
        deltaphi_bb = np.abs(m[np.triu_indices(len(m[0]), k = 1)])
        deltaphi_bb = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_bb])
        
        if min(f(deltaeta_bb, deltaphi_bb)) > 0.4:
            event_deltaR_bb.append(i)
            
    event_deltaR_ll = []
    for i in tqdm(event_deltaR_bb):
        lep_eta, lep_phi = lepton_eta[i], lepton_phi[i]
        m = lep_eta - lep_eta[:, np.newaxis]
        deltaeta_ll = m[np.triu_indices(len(m[0]), k = 1)]
        m = lep_phi - lep_phi[:, np.newaxis]
        deltaphi_ll = np.abs(m[np.triu_indices(len(m[0]), k = 1)])
        deltaphi_ll = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_ll])

        if min(f(deltaeta_ll, deltaphi_ll)) > 0.4:
            event_deltaR_ll.append(i)
    
    # HT of 2 leading leptons and three leading jets
    event_ht = []
    for i in tqdm(event_deltaR_ll):
        lep_pt, jet_pt = lepton_sort_pt[i], jet_sort_pt[i]
        ht = np.sum([lep_pt[0], lep_pt[1], lep_pt[2], jet_pt[0], jet_pt[1], jet_pt[2]])
        if ht > 320:
            event_ht.append(i)
                
    
    cs_b = cs * 1000
    sel_eff = len(event_ht)/EventSize
    uncertainty = np.sqrt(sel_eff * (1-sel_eff)/ EventSize)
    cs_f = cs_b * len(event_ht)/EventSize
                
    return [[EventSize, len(event_signal), len(event_lep), len(event_jet), len(event_emiss), len(event_deltaR_bl), len(event_deltaR_bb), len(event_deltaR_ll), len(event_ht)], [cs_b, sel_eff, cs_f, uncertainty]]

In [3]:
y1 = uproot.open('../root/MG5_aMC_v2_7_3/ttW/Events/run_03/tag_1_delphes_events.root')
y2 = uproot.open('../root/MG5_aMC_v2_7_3/ttZ/Events/run_02/tag_1_delphes_events.root')
y3 = uproot.open('../root/MG5_aMC_v2_7_3/tZj/Events/run_01/tag_1_delphes_events.root')
ttW = function(y1, 0.04497)
ttZ = function(y2, 0.01659)
tZj = function(y3, 0.01289)

100%|██████████| 2/2 [00:00<00:00, 831.46it/s]


In [10]:
name = [r'Input Event Size',
        '3b3l Signature',
        r'Jet Selection',
        r'lepton Selection',
        r'$E_{T}^{miss}$ > 30GeV',
        r'Δ$R_{bl}$ > 0.4',
        r'Δ$R_{bb}$ > 0.4',
        r'Δ$R_{ll}$ > 0.4',
        r'$H_{T}$ Cut']
fd = pd.DataFrame({'Selection Cut (3b3l)': name,
                   'ttW+j': ttW[0],
                   'ttZ+j': ttZ[0],
                   'tZj': tZj[0],
                   })
fd

,Selection Cut (3b3l),ttW+j,ttZ+j,tZj
0,Input Event Size,10000,5000,5000
1,3b3l Signature,29,44,6
2,Jet Selection,15,36,3
3,lepton Selection,15,34,2
4,$E_{T}^{miss}$ > 30GeV,15,28,2
5,Δ$R_{bl}$ > 0.4,15,28,2
6,Δ$R_{bb}$ > 0.4,15,28,2
7,Δ$R_{ll}$ > 0.4,15,22,2
8,$H_{T}$ Cut,15,22,2


In [11]:
name2 = ['Cross Section from MadGraph Generation (fb)',
         'Selection Efficiency',
         'Cross Section after Selection Cut (fb)',
         'Uncertainty']
fd = pd.DataFrame({'Selection Cut (3b3l)': name2,
                   'ttW+j': ttW[1],
                   'ttZ+j': ttZ[1],
                   'tZj': tZj[1],
                   })
fd

,Selection Cut (3b3l),ttW+j,ttZ+j,tZj
0,Cross Section from MadGraph Generation (fb),44.970000,16.590000,12.890000
1,Selection Efficiency,0.001500,0.004400,0.000400
2,Cross Section after Selection Cut (fb),0.067455,0.072996,0.005156
3,Uncertainty,0.000387,0.000936,0.000283


In [13]:
name = [
     'ttW+j',
     'ttZ+j',
     'tZ+j',
]
n1 = [0.0017, 0.0205, 0.0002]
n2 = [ttW[-1][-2], ttZ[-1][-2], tZj[-1][-2]]
uf = [ttW[1][-1], ttZ[1][-1], tZj[1][-1]]
fd = pd.DataFrame({r'$M_{H}$': name,
    'Signal Cross Section in Paper (fb)': n1,
                   'My Result (fb)': n2,
                   'Uncertainty': uf
                   })
fd

,$M_{H}$,Signal Cross Section in Paper (fb),My Result (fb),Uncertainty
0,ttW+j,0.0017,0.067455,0.000387
1,ttZ+j,0.0205,0.072996,0.000936
2,tZ+j,0.0002,0.005156,0.000283
